# West Nile Virus

## Feature Engineering

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
df = pd.read_csv('/Users/ultimateflexer/Desktop/Datasets/predict-west-nile-virus/clean_df.csv')

In [4]:
df.head(10)

,Date,Address,Street,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,...,ResultSpeed_lag_14,ResultSpeed_lag_21,ResultDir_lag_1,ResultDir_lag_7,ResultDir_lag_14,ResultDir_lag_21,AvgSpeed_lag_1,AvgSpeed_lag_7,AvgSpeed_lag_14,AvgSpeed_lag_21
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,T002,41.954690,-87.800991,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,T002,41.954690,-87.800991,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",N MANDELL AVE,T007,41.994991,-87.769279,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",W FOSTER AVE,T015,41.974089,-87.824812,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",W FOSTER AVE,T015,41.974089,-87.824812,4,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
5,2007-05-29,"1500 West Webster Avenue, Chicago, IL 60614, USA",W WEBSTER AVE,T045,41.921600,-87.666455,2,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
6,2007-05-29,"2500 West Grand Avenue, Chicago, IL 60654, USA",W GRAND AVE,T046,41.891118,-87.654491,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
7,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",W ROOSEVELT,T048,41.867108,-87.654224,1,0,0,0,...,NaN,NaN,17.5,17.5,NaN,NaN,9.0,9.0,NaN,NaN
8,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",W ROOSEVELT,T048,41.867108,-87.654224,2,0,0,0,...,NaN,NaN,17.5,17.5,NaN,NaN,9.0,9.0,NaN,NaN
9,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",W CHICAGO,T049,41.896282,-87.655232,1,0,0,0,...,NaN,NaN,17.5,17.5,NaN,NaN,9.0,9.0,NaN,NaN


In [5]:
# Before I perform information value analysis on the data, I am going to drop the address 
# columns as well as the trap column. I also need to deal with NaN values. These will be dropped
# so as not to interfere with the lag variables. 

df.set_index('Date', inplace=True)
df.drop(['Address', 'Street', 'Trap'], axis=1, inplace=True)

df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10392 entries, 2007-05-29 to 2013-09-26
Data columns (total 87 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Latitude                        10392 non-null  float64
 1   Longitude                       10392 non-null  float64
 2   NumMosquitos                    10392 non-null  int64  
 3   WnvPresent                      10392 non-null  int64  
 4   Species_CULEX ERRATICUS         10392 non-null  int64  
 5   Species_CULEX PIPIENS           10392 non-null  int64  
 6   Species_CULEX PIPIENS/RESTUANS  10392 non-null  int64  
 7   Species_CULEX RESTUANS          10392 non-null  int64  
 8   Species_CULEX SALINARIUS        10392 non-null  int64  
 9   Species_CULEX TARSALIS          10392 non-null  int64  
 10  Species_CULEX TERRITANS         10392 non-null  int64  
 11  Tmax                            10392 non-null  float64
 12  Tmin                   

In [6]:
# Split data into train and test sets

y = df['WnvPresent']
X = df.drop('WnvPresent', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state=42, stratify=y)

In [7]:
# Information Value Analysis via the Variance Inlfation Factor.

def iterate_vif(df, vif_threshold=5, max_vif=6):
    count = 0
    while max_vif > vif_threshold:
        count += 1
        print("Iteration # "+str(count))
        vif = pd.DataFrame()
        vif["VIFactor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        vif["features"] = df.columns

        if vif['VIFactor'].max() > vif_threshold:
            print('Removing %s with VIF of %f' % (vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], vif['VIFactor'].max()))
            df = df.drop(vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], axis=1)
            max_vif = vif['VIFactor'].max()
        else:
            print('Complete')
            break
    return df, vif.sort_values('VIFactor')

final_df, final_vif = iterate_vif(X_train)

Iteration # 1


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Removing Species_CULEX ERRATICUS with VIF of inf
Iteration # 2


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Removing Tavg with VIF of inf
Iteration # 3


...

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


Removing Species_CULEX PIPIENS/RESTUANS with VIF of 9.935017
Iteration # 63


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


Removing ResultSpeed_lag_21 with VIF of 5.441843
Iteration # 64
Complete


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


In [8]:
final_df.head()

,NumMosquitos,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Heat,Depth,SnowFall,PrecipTotal,...,Cool_lag_21,Depth_lag_1,Depth_lag_7,Depth_lag_14,Depth_lag_21,SnowFall_lag_1,SnowFall_lag_7,SnowFall_lag_14,SnowFall_lag_21,PrecipTotal_lag_21
Date,,,,,,,,,,,,,,,,,,,,,
2013-07-25,6,0,0,0,0,0,0.0,0.0,0.0,0.0,...,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-09-26,5,0,0,0,0,0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-08-12,2,0,1,0,0,0,0.0,0.0,0.0,0.0,...,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2007-07-11,6,0,0,0,0,0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-08-08,2,1,0,0,0,0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
final_vif

,VIFactor,features
4,1.007717,Species_CULEX TARSALIS
3,1.020086,Species_CULEX SALINARIUS
5,1.058026,Species_CULEX TERRITANS
1,1.500469,Species_CULEX PIPIENS
2,1.512638,Species_CULEX RESTUANS
0,1.705421,NumMosquitos
9,3.735686,PrecipTotal
22,3.794456,PrecipTotal_lag_21
10,4.090379,ResultDir
13,4.209629,Cool_lag_21


In [10]:
final_vif.reset_index(drop=True, inplace=True)
final_vif

,VIFactor,features
0,1.007717,Species_CULEX TARSALIS
1,1.020086,Species_CULEX SALINARIUS
2,1.058026,Species_CULEX TERRITANS
3,1.500469,Species_CULEX PIPIENS
4,1.512638,Species_CULEX RESTUANS
5,1.705421,NumMosquitos
6,3.735686,PrecipTotal
7,3.794456,PrecipTotal_lag_21
8,4.090379,ResultDir
9,4.209629,Cool_lag_21


In [11]:
final_df.describe()

,NumMosquitos,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Heat,Depth,SnowFall,PrecipTotal,...,Cool_lag_21,Depth_lag_1,Depth_lag_7,Depth_lag_14,Depth_lag_21,SnowFall_lag_1,SnowFall_lag_7,SnowFall_lag_14,SnowFall_lag_21,PrecipTotal_lag_21
count,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.0,8313.0,8313.000000,...,8313.000000,8313.0,8313.0,8313.0,8313.0,8313.0,8313.0,8313.0,8313.0,8313.000000
mean,13.055936,0.254782,0.258390,0.008421,0.000722,0.022134,0.864910,0.0,0.0,0.123172,...,8.937628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.123713
std,16.266714,0.435765,0.437776,0.091382,0.026858,0.147128,2.659059,0.0,0.0,0.240267,...,5.744583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.240532
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
50%,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,9.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
75%,18.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.100000,...,14.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000
max,50.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14.000000,0.0,0.0,1.235000,...,20.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.235000


In [12]:
# It appears some of these columns are entirely 0 and of no use. Let's narrow it down to
# the important stuff.

final_df.drop(['Depth', 'SnowFall', 'Depth_lag_1', 'Depth_lag_7', 'Depth_lag_14', 'Depth_lag_21',
              'SnowFall_lag_1', 'SnowFall_lag_7', 'SnowFall_lag_14', 'SnowFall_lag_21'],
              axis=1, inplace=True)
final_vif.drop(final_vif.index[13:23], axis=0, inplace=True)

In [13]:
final_df.head()

,NumMosquitos,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Heat,PrecipTotal,ResultDir,Region,Heat_lag_21,Cool_lag_21,PrecipTotal_lag_21
Date,,,,,,,,,,,,,
2013-07-25,6,0,0,0,0,0,0.0,0.0,23.0,5,0.0,4.5,0.0
2013-09-26,5,0,0,0,0,0,0.0,0.0,11.0,3,0.0,1.0,0.0
2011-08-12,2,0,1,0,0,0,0.0,0.0,19.0,8,0.0,7.5,0.0
2007-07-11,6,0,0,0,0,0,0.0,0.0,30.0,5,0.0,2.0,0.0
2013-08-08,2,1,0,0,0,0,0.0,0.0,5.0,5,0.0,8.0,0.0


In [14]:
final_vif

,VIFactor,features
0,1.007717,Species_CULEX TARSALIS
1,1.020086,Species_CULEX SALINARIUS
2,1.058026,Species_CULEX TERRITANS
3,1.500469,Species_CULEX PIPIENS
4,1.512638,Species_CULEX RESTUANS
5,1.705421,NumMosquitos
6,3.735686,PrecipTotal
7,3.794456,PrecipTotal_lag_21
8,4.090379,ResultDir
9,4.209629,Cool_lag_21


In [15]:
final_df.sort_index(inplace=True)
final_df.head(10)

,NumMosquitos,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Heat,PrecipTotal,ResultDir,Region,Heat_lag_21,Cool_lag_21,PrecipTotal_lag_21
Date,,,,,,,,,,,,,
2007-05-29,1,0,0,0,0,0,0.0,0.00,17.5,7,0.0,12.0,0.00
2007-05-29,1,0,1,0,0,0,0.0,0.00,17.5,3,0.0,12.0,0.00
2007-05-29,1,0,1,0,0,0,0.0,0.00,17.5,4,0.0,12.0,0.00
2007-06-05,3,0,1,0,0,0,7.5,0.14,10.0,6,7.5,0.0,0.14
2007-06-05,1,0,0,0,0,0,7.5,0.14,10.0,7,7.5,0.0,0.14
2007-06-05,1,0,1,0,0,0,7.5,0.14,10.0,7,7.5,0.0,0.14
2007-06-05,1,1,0,0,0,0,7.5,0.14,10.0,2,7.5,0.0,0.14
2007-06-05,1,0,1,0,0,0,7.5,0.14,10.0,5,0.0,12.0,0.00
2007-06-05,1,0,1,0,0,0,7.5,0.14,10.0,5,0.0,12.0,0.00


In [18]:
# As the final pre-processing step, I will subset X_train and X_test on final_df, which 
# shows which columns are of use.

X_train = X_train[final_df.columns]
X_train.columns

Index(['NumMosquitos', 'Species_CULEX PIPIENS', 'Species_CULEX RESTUANS',
       'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS',
       'Species_CULEX TERRITANS', 'Heat', 'PrecipTotal', 'ResultDir', 'Region',
       'Heat_lag_21', 'Cool_lag_21', 'PrecipTotal_lag_21'],
      dtype='object')

In [19]:
X_test = X_test[final_df.columns]
X_test.columns

Index(['NumMosquitos', 'Species_CULEX PIPIENS', 'Species_CULEX RESTUANS',
       'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS',
       'Species_CULEX TERRITANS', 'Heat', 'PrecipTotal', 'ResultDir', 'Region',
       'Heat_lag_21', 'Cool_lag_21', 'PrecipTotal_lag_21'],
      dtype='object')

In [20]:
# Scale data.
from sklearn import preprocessing

# Build scaler based on training data and apply it to test data to then also scale the test data.
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Modeling

Now that the data is prepped for modeling, it is time to implement these models. I will be using 3 different models, and determining which is the best fit for predicting the target feature, Wnv_Present. These models will include a random forest model, a ridge regression model, and an XG boost model.